In [ ]:
import numpy as np

In [ ]:
class GMM:
    """
    Gaussian Mixture Model.
    
    Parameters
    -------
        data: np.ndarray
            A (D x N) matrix containing data.
        
        init_mweights_vector: np.ndarray
            A (K x 1) vector containing K Gaussian mixture weights.
        
        init_mean_vectors: np.ndarray
            A 2D (D x K) matrix, whose K columns are K mean vectors.
            
        init_cov_matrices: np.ndarray
            A 3D (K x D x D) matrix, in which the first dimension contains K cov. matrices.
    
    Methods
    -------
        _validate_params():
            Checks the validity of given arguments.
            Parameters:
                data: np.ndarray
                init_mweights_vector: np.ndarray
                init_mean_vectors: np.ndarray
                init_cov_matrices: np.ndarray
            Return: None
        
        _calc_pdf():
            Computes the value of Gaussian pdf according to given parameters,
                pdf = (2*pi)^(-D/2) * det(sigma)^(-1/2) * exp( (-1/2) * (x - mu).T @ sigma^-1 @ (x - mu) )
            Parameters:
                data_point: np.ndarray
                    A (D x 1) vector representing a data point.
                mean: np.ndarray
                    A (D x 1) mean vector.
                covariance: np.ndarray
                    A (D x D) covariance matrix.
            Return: float
        
        _calc_resp():
            Computes the responsibilities of a mixture component on a data point,
                r_nk = pi_k * N(x_n | mu_k, sigma_k) / sum(pi_j * N(x_n | mu_j, sigma_j)), j = 1, 2, ..., K
            Parameters:
                data_point: np.ndarray
            Return: np.ndarray
                A (K x 1) vector, whose entries are the resp. of the k-th component on the data point.
        
        _calc_total_resp():
            Computes the total responsiblities of a mixture component on every data point,
                N_k = sum(r_nk), n = 1, 2, ..., N
            Parameters:
                k_mcomponent: int
                    The k-th mixture component.
            Return: 
                total_resp: float
                resp_vector: np.ndarray
                    A (D x 1) vector containing the resp. of k-th component on every data point.
        
        _update_mean_vectors():
            Updates mean vectors of mixture components according to the current value of resp.
            Parameters: None
            Return: None
        
        _update_cov_matrices():
            Updates cov. matrices of mixture components according to new mean vectors and current value of resp.
            Parameters: None
            Return: None
        
        _update_mweights_vector():
            Updates mixture weights vector.
            Parameters: None
            Return: None
            
        EM():
            Performs EM algorithm
                E-step: Evaluates resp. matrix of every mixture component on every data point.
                M-step: Updates the means, covariances, and mixture weights.
            Parameters: 
                n_iterations: int
                    The number of iterations
            Return: None
    """
    def __init__(self, data, init_mweights_vector, init_mean_vectors, init_cov_matrices):
        self._validate_params(data, init_mweights, init_means, init_covariances)
        self.data = data
        self.mweights_vector = init_mweights_vector
        self.mean_vectors = init_mean_vectors
        self.cov_matrices = init_cov_matrices
        self.n_mcomponents = len(init_mweights_vector)
        self.resp_matrix = None
        
    def _validate_params(self, data, init_mweights_vector, 
                         init_mean_vectors, init_cov_matrices):
        pass
        
    def _calc_pdf(self, data_point: np.ndarray, mean: np.ndarray, covariance: np.ndarray) -> float:
        std = np.linalg.det(covariance)**(1/2)
        constant_term = (2*np.pi)**(data_point.size/2) * (1/std)
        exp_term = (data_point - mean).T @ np.linalg.inv(covariance) @ (data_point - mean)
        pdf = constant_term * np.exp((-1/2) * exp_term)
        return pdf
    
    def _calc_resp(self, data_point: np.ndarray) -> np.ndarray:
        # Initialize paramaters
        mweights_vector = self.mweights_vector
        mean_vectors = self.means
        cov_matrices = self.covariances
        n_mcomponents = self.n_mcomponents
        resp = mweights_vector
        
        # Calculate mixture-weighted sum of all Gaussians
        mcomponents_total = 0
        for i in range(n_mcomponents):
            mcomponent_pdf = self._calc_pdf(data_point, mean_vectors[:, i], cov_matrices[i, :, :])
            resp[i] *= mcomponent_pdf
            mcomponents_total += mweights_vector[i] * mcomponent_pdf
        
        return resp / mcomponents_total
    
    def _calc_total_resp(self) -> None:
        data = self.data
        self.resp_matrix = np.apply_along_axis(self._calc_resp, axis=0, arr=data)
        
    def _update_mean_vectors(self) -> None:
        # Initialize parameters
        data = self.data
        n_mcomponents = self.n_mcomponents
        resp_matrix = self.resp_matrix
        
        # Compute new mean vectors 
        upd_mean_vectors = self.mean_vectors
        for i in range(n_mcomponents):
            total_resp, resp_vector = resp_matrix[i, :].sum(), resp_matrix[i, :]
            resp_weighted_data =  np.sum(resp_vector * data, axis=1)
            upd_mean_vectors[:, i] = (1/total_resp) * resp_weighted_data
            
        self.mean_vectors = upd_mean_vectors
        
    def _update_cov_matrices(self) -> None:
        # Initialize parameters
        data = self.data
        n_mcomponents = self.n_mcomponents
        mean_vectors = self.mean_vectors
        resp_matrix = self.resp_matrix
        
        # Compute new cov. matrices
        upd_cov_matrices = self.cov_matrices
        for i in range(n_mcomponents):
            total_resp, resp_vector = resp_matrix[i, :].sum(), resp_matrix[i, :]
            resp_weighted_data = ((resp_vector*data) @ data.T) \
                                - (np.sum(resp_vector*data, axis=1) @ mean_vectors[:, i].T) \
                                - (mean_vectors[:, i] @ np.sum(resp_vector*data, axis=1).T) \
                                + (resp_vector.sum() * (mean_vectors[:, i] @ mean_vectors[:, i].T))
            upd_cov_matrices[i, :, :] = (1/total_resp) * resp_weight_data
        
        self.cov_matrices = upd_cov_matrices
            
    def _update_mweights_vector(self) -> None:
        # Initialize parameters
        n_datapoints = self.data.shape[-1]
        n_mcomponents = self.n_mcomponents
        resp_matrix = self.resp_matrix
        
        # Compute mixture responsibilities
        total_resp_vector = resp_matrix.sum(axis=1)        
        
        # Compute new mixture weights vector
        self.mweights_vector = total_resp_vector / n_datapoints
        
    def EM(self, n_iterations: int) -> None:
        while n_iterations > 0:
            self._calc_total_resp
            self._update_mean_vectors()
            self._update_cov_matrices()
            self._update_mweights_vector()
            n_iterations -= 1